In [1]:
import json
from textwrap import dedent

import pytablereader as ptr
import pytablewriter as ptw

In [2]:
ptr.TableFileLoader.get_format_name_list()

['csv',
 'excel',
 'html',
 'json',
 'ltsv',
 'markdown',
 'mediawiki',
 'sqlite',
 'tsv']

In [3]:
ptr.TableUrlLoader.get_format_name_list()

['csv',
 'excel',
 'html',
 'json',
 'ltsv',
 'markdown',
 'mediawiki',
 'sqlite',
 'tsv']

In [4]:
loader = ptr.CsvTableTextLoader(
    "\n".join([
        "a,b",
        "1,2",
        "3.3,4.4",
    ]))
for table_data in loader.load():
    print(ptw.dump_tabledata(table_data, format_name="markdown"))


# csv1
| a | b |
|--:|--:|
|1.0|2.0|
|3.3|4.4|




In [5]:
json_list = [
    dedent("""\
        [
            {"attr_b": 4, "attr_c": "a", "attr_a": 1},
            {"attr_b": 2.1, "attr_c": "bb", "attr_a": 2},
            {"attr_b": 120.9, "attr_c": "ccc", "attr_a": 3}
        ]"""),
    dedent("""\
        {
            "attr_a": [1, 2, 3],
            "attr_b": [4, 2.1, 120.9],
            "attr_c": ["a", "bb", "ccc"]
        }"""),
    dedent("""\
        {
            "num_ratings": 27,
            "support_threads": 1,
            "downloaded": 925716,
            "last_updated":"2017-12-01 6:22am GMT",
            "added":"2010-01-20",
            "num": 1.1,
            "hoge": null
        }"""),
    dedent("""\
        {
            "table_a" : [
                {"attr_b": 4, "attr_c": "a", "attr_a": 1},
                {"attr_b": 2.1, "attr_c": "bb", "attr_a": 2},
                {"attr_b": 120.9, "attr_c": "ccc", "attr_a": 3}
            ],
            "table_b" : [
                {"a": 1, "b": 4},
                {"a": 2 },
                {"a": 3, "b": 120.9}
            ]
        }"""),
    dedent("""\
        {
            "table_a" : {
                "attr_a": [1, 2, 3],
                "attr_b": [4, 2.1, 120.9],
                "attr_c": ["a", "bb", "ccc"]
            },
            "table_b" : {
                "a": [1, 3],
                "b": [4, 120.9]
            }
        }"""),
    dedent("""\
        {
            "table_a": {
                "num_ratings": 27,
                "support_threads": 1,
                "downloaded": 925716,
                "last_updated":"2017-12-01 6:22am GMT",
                "added":"2010-01-20",
                "num": 1.1,
                "hoge": null
            },
            "table_b": {
                "a": 4,
                "b": 120.9
            }
        }"""),
]

for i, data in enumerate(json_list):
    print("# {} ----------------------------------------".format(i + 1))
    loader = ptr.JsonTableTextLoader(data)

    for table_data in loader.load():
        #table_data.table_name = ""
        print(ptw.dump_tabledata(table_data, format_name="rst"))


# 1 ----------------------------------------
.. table:: json1

    +------+------+------+
    |attr_a|attr_b|attr_c|
    +======+======+======+
    |     1|   4.0|a     |
    +------+------+------+
    |     2|   2.1|bb    |
    +------+------+------+
    |     3| 120.9|ccc   |
    +------+------+------+


# 2 ----------------------------------------
.. table:: json2

    +------+------+------+
    |attr_a|attr_b|attr_c|
    +======+======+======+
    |     1|   4.0|a     |
    +------+------+------+
    |     2|   2.1|bb    |
    +------+------+------+
    |     3| 120.9|ccc   |
    +------+------+------+


# 3 ----------------------------------------
.. table:: json3

    +---------------+---------------------+
    |      key      |        value        |
    +===============+=====================+
    |num_ratings    |                   27|
    +---------------+---------------------+
    |support_threads|                    1|
    +---------------+---------------------+
    |download

In [6]:
from pytablereader.json.formatter import (
    MultipleJsonTableConverterA,
    MultipleJsonTableConverterB,
    MultipleJsonTableConverterC,
    SingleJsonTableConverterA,
    SingleJsonTableConverterB,
    SingleJsonTableConverterC,
)

converter_class_list = (
    MultipleJsonTableConverterA,
    MultipleJsonTableConverterB,
    MultipleJsonTableConverterC,
    SingleJsonTableConverterA,
    SingleJsonTableConverterB,
    SingleJsonTableConverterC,
)

for converter_class in converter_class_list:
    print("#JSON schema: {}\n{}\n".format(
        converter_class.__name__, json.dumps(converter_class(None)._schema, indent=4)))


#JSON schema: MultipleJsonTableConverterA
{
    "type": "object",
    "additionalProperties": {
        "type": "array",
        "items": {
            "type": "object",
            "additionalProperties": {
                "anyOf": [
                    {
                        "type": "string"
                    },
                    {
                        "type": "number"
                    },
                    {
                        "type": "boolean"
                    },
                    {
                        "type": "null"
                    }
                ]
            }
        }
    }
}

#JSON schema: MultipleJsonTableConverterB
{
    "type": "object",
    "additionalProperties": {
        "type": "object",
        "additionalProperties": {
            "type": "array",
            "items": {
                "anyOf": [
                    {
                        "type": "string"
                    },
                    {
                        "type

In [7]:
import json
import jsonschema
from collections import OrderedDict

value = """\
{
    "num_ratings": 27,
    "support_threads": 1,
    "support_threads_resolved": 0,
    "downloaded": 925716,
    "last_updated":"2017-12-01 6:22am GMT",
    "added":"2010-01-20",
    "num": 1.1,
    "hoge": null
}"""

schema = {
    "type": "object",
    "additionalProperties": {
        "anyOf": [
            {"type": "string"},
            {"type": "number"},
            {"type": "boolean"},
            {"type": "null"},
        ],
    },
}

jsonschema.validate(json.loads(value), schema)

json_data = json.loads(value, object_pairs_hook=OrderedDict)
for entry in json_data.items():
    print(entry)


('num_ratings', 27)
('support_threads', 1)
('support_threads_resolved', 0)
('downloaded', 925716)
('last_updated', '2017-12-01 6:22am GMT')
('added', '2010-01-20')
('num', 1.1)
('hoge', None)
